![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/healthcare-nlp/08.1.Text_Classification_with_DocumentMLClassifier.ipynb)

# Colab Setup

In [ ]:
! pip install -q johnsnowlabs

In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

In [ ]:
from johnsnowlabs import nlp, medical

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
nlp.install()

In [4]:
from johnsnowlabs import nlp, medical, visual
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import pyspark.sql.functions as F

# Automatically load license data and start a session with all jars user has access to
spark = nlp.start(hardware_target="cpu")
spark

👌 Detected license file /content/spark_nlp_for_healthcare_spark_ocr_10494 (1).json
👌 Launched cpu optimized session with with: 🚀Spark-NLP==6.1.3, 💊Spark-Healthcare==6.1.1, running on ⚡ PySpark==3.4.0


## Healthcare NLP for Data Scientists Course

If you are not familiar with the components in this notebook, you can check [Healthcare NLP for Data Scientists Udemy Course](https://www.udemy.com/course/healthcare-nlp-for-data-scientists/) and the [MOOC Notebooks](https://github.com/JohnSnowLabs/spark-nlp-workshop/tree/master/Spark_NLP_Udemy_MOOC/Healthcare_NLP) for each components.

## 🔎 Models :


|index|model|
|-----:|:-----|
|1|[classifierml_ade](https://nlp.johnsnowlabs.com/2023/05/16/classifierml_ade_en.html)


## Load ADE Classification Dataset

In [5]:
#downloading sample datasets
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/ADE_Corpus_V2/ADE-NEG.txt
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/ADE_Corpus_V2/DRUG-AE.rel

**ADE-Negative Dataset**

In [6]:
df_neg= pd.read_csv("ADE-NEG.txt", header=None, delimiter="\t", names=["col1"])
df_neg['text'] =  df_neg.col1.str.split('NEG').str[1]
df_neg["category"] = "neg"
df_neg= df_neg[["text", "category"]]
df_neg.head()

,text,category
0,Clioquinol intoxication occurring in the trea...,neg
1,"""Retinoic acid syndrome"" was prevented with s...",neg
2,BACKGROUND: External beam radiation therapy o...,neg
3,"Although the enuresis ceased, she developed t...",neg
4,A 42-year-old woman had uneventful bilateral ...,neg


**ADE-Positive Dataset**

In [7]:
df_pos= pd.read_csv("DRUG-AE.rel", header=None, delimiter="|")
df_pos["category"]= "pos"
df_pos.rename(columns={1: "text"}, inplace=True)
df_pos= df_pos[["text", "category"]]
df_pos.head()

,text,category
0,Intravenous azithromycin-induced ototoxicity.,pos
1,"Immobilization, while Paget's bone disease was...",pos
2,Unaccountable severe hypercalcemia in a patien...,pos
3,METHODS: We report two cases of pseudoporphyri...,pos
4,METHODS: We report two cases of pseudoporphyri...,pos


**Merging positive and negative datasets**

In [8]:
ade_df= pd.concat([df_neg, df_pos]).sample(frac=1) #merge and shuffle the data
ade_df.head()

,text,category
14890,Herpetic tracheobronchitis.,neg
2664,We present a case of hemolytic-uremic syndrome...,pos
1916,Diagnosis had been delayed secondary to sampl...,neg
15880,Gastric stump lymphoma after distal gastrecto...,neg
13002,Lithium has been reported to raise serum calc...,neg


In [9]:
ade_df["category"].value_counts()

,count
category,
neg,16695
pos,6821


In [10]:
ade_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23516 entries, 14890 to 3345
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      23516 non-null  object
 1   category  23516 non-null  object
dtypes: object(2)
memory usage: 551.2+ KB


In [11]:
spark_df = spark.createDataFrame(ade_df)
spark_df.head(3)

[Row(text=' Herpetic tracheobronchitis.', category='neg'),
 Row(text='We present a case of hemolytic-uremic syndrome that developed during the 4th cycle of combination chemotherapy with oxaliplatin, 5-fluorouracil and leucovorin.', category='pos'),
 Row(text=' Diagnosis had been delayed secondary to sampling of the other uterine horn.', category='neg')]

In [12]:
spark_df.groupBy("category").count().show()
spark_df.printSchema()

+--------+-----+
|category|count|
+--------+-----+
|     pos| 6821|
|     neg|16695|
+--------+-----+

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)



**Get train & test set**

In [13]:
train_data, test_data = spark_df.randomSplit([0.8, 0.2], seed = 100)

print("Train Dataset Count: " + str(train_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Train Dataset Count: 18793
Test Dataset Count: 4723


## DocumentMLClassifier with Linear SVM

The below pipeline uses the SVM classifier of `DocumentMLClassifier` with setting `setClassificationModelClass("svm")`.

In [14]:
document = nlp.DocumentAssembler().setInputCol("text").setOutputCol("document")

token = nlp.Tokenizer().setInputCols("document").setOutputCol("token")

stemmer = nlp.Stemmer() \
    .setInputCols(["token"]) \
    .setOutputCol("stem")

classifier_svm = medical.DocumentMLClassifierApproach() \
    .setInputCols("stem") \
    .setLabelCol("category") \
    .setOutputCol("prediction") \
    .setClassificationModelClass("svm")

pipeline = nlp.Pipeline(stages=[document, token, stemmer, classifier_svm])

In [15]:
svm_model = pipeline.fit(train_data)
result_svm = svm_model.transform(test_data).cache()
result_svm.show()

+--------------------+--------+--------------------+--------------------+--------------------+--------------------+
|                text|category|            document|               token|                stem|          prediction|
+--------------------+--------+--------------------+--------------------+--------------------+--------------------+
| 8 months later s...|     neg|[{document, 0, 17...|[{token, 1, 1, 8,...|[{token, 1, 1, 8,...|[{category, 1, 17...|
| A 27-year-old Ca...|     neg|[{document, 0, 11...|[{token, 1, 1, A,...|[{token, 1, 1, a,...|[{category, 1, 11...|
| A 37 year woman ...|     neg|[{document, 0, 11...|[{token, 1, 1, A,...|[{token, 1, 1, a,...|[{category, 1, 11...|
| A 5-year-old, sp...|     neg|[{document, 0, 13...|[{token, 1, 1, A,...|[{token, 1, 1, a,...|[{category, 1, 13...|
| A 55-year-old ma...|     neg|[{document, 0, 20...|[{token, 1, 1, A,...|[{token, 1, 1, a,...|[{category, 1, 20...|
| A 69-year-old ma...|     neg|[{document, 0, 18...|[{token, 1, 1, A,...

In [16]:
result_svm.printSchema()

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- token: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- va

In [17]:
result_svm.select("text","prediction").show(4, truncate=100)

+----------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
|                                                                                                text|                                                                      prediction|
+----------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
| 8 months later she was scheduled for salvage chemotherapy with gemcitabine (1,000 mg/m(2) days 1...|[{category, 1, 172, neg, {sentence -> 0, chunk -> 0, confidence -> 0.9165}, []}]|
| A 27-year-old Caucasian woman with end-stage Crohn's disease was hospitalized with an upper extr...|[{category, 1, 113, neg, {sentence -> 0, chunk -> 0, confidence -> 0.8796}, []}]|
| A 37 year woman developed neuroleptic induced parkinsonism that persisted long

In [18]:
result_svm_df = result_svm.select('category','prediction.result').toPandas()
result_svm_df['result'] = result_svm_df.result.apply(lambda x : x[0])

print (classification_report(result_svm_df['category'], result_svm_df['result']))

              precision    recall  f1-score   support

         neg       0.90      0.90      0.90      3383
         pos       0.75      0.75      0.75      1340

    accuracy                           0.86      4723
   macro avg       0.82      0.83      0.83      4723
weighted avg       0.86      0.86      0.86      4723



## DocumentMLClassifier with Logistic Regression

In [19]:
classifier_logreg = medical.DocumentMLClassifierApproach() \
    .setInputCols("token") \
    .setLabelCol("category") \
    .setOutputCol("prediction") \
    .setClassificationModelClass("logreg")\

pipeline = nlp.Pipeline(stages=[document, token, classifier_logreg])

logreg_model = pipeline.fit(train_data)
result_logreg = logreg_model.transform(test_data).cache()

result_logreg_df = result_logreg.select('category','prediction.result').toPandas()
result_logreg_df['result'] = result_logreg_df.result.apply(lambda x : x[0])

print (classification_report(result_logreg_df['category'], result_logreg_df['result']))

              precision    recall  f1-score   support

         neg       0.90      0.92      0.91      3383
         pos       0.78      0.74      0.76      1340

    accuracy                           0.87      4723
   macro avg       0.84      0.83      0.83      4723
weighted avg       0.87      0.87      0.87      4723



## Playing with Some Parameters

In [20]:
classifier_logreg.extractParamMap()

{Param(parent='DocumentMLClassifierApproach_1634b83a5b7e', name='lazyAnnotator', doc='Whether this AnnotatorModel acts as lazy in RecursivePipelines'): False,
 Param(parent='DocumentMLClassifierApproach_1634b83a5b7e', name='labelCol', doc='column with the value result we are trying to predict.'): 'category',
 Param(parent='DocumentMLClassifierApproach_1634b83a5b7e', name='maxIter', doc='maximum number of iterations.'): 10,
 Param(parent='DocumentMLClassifierApproach_1634b83a5b7e', name='tol', doc='convergence tolerance after each iteration.'): 1e-06,
 Param(parent='DocumentMLClassifierApproach_1634b83a5b7e', name='fitIntercept', doc='whether to fit an intercept term, default is true.'): True,
 Param(parent='DocumentMLClassifierApproach_1634b83a5b7e', name='vectorizationModelPath', doc='specify the vectorization model if it has been already trained.'): '',
 Param(parent='DocumentMLClassifierApproach_1634b83a5b7e', name='classificationModelPath', doc='specify the classification model if 

**Change `maxIter` parameter with `setMaxIter()`**

In [21]:
classifier_logreg = medical.DocumentMLClassifierApproach() \
    .setInputCols("token") \
    .setLabelCol("category") \
    .setOutputCol("prediction") \
    .setClassificationModelClass("logreg")\
    .setMaxIter(5)

pipeline = nlp.Pipeline(stages=[document, token, classifier_logreg])

result_logreg = pipeline.fit(train_data).transform(test_data).cache()

result_logreg_df = result_logreg.select('category','prediction.result').toPandas()
result_logreg_df['result'] = result_logreg_df.result.apply(lambda x : x[0])

print (classification_report(result_logreg_df['category'], result_logreg_df['result']))

              precision    recall  f1-score   support

         neg       0.89      0.95      0.92      3383
         pos       0.85      0.71      0.77      1340

    accuracy                           0.88      4723
   macro avg       0.87      0.83      0.85      4723
weighted avg       0.88      0.88      0.88      4723



**Change `tol` (tolarance) parameter with `setTol()`**

In [22]:
classifier_logreg = medical.DocumentMLClassifierApproach() \
    .setInputCols("token") \
    .setLabelCol("category") \
    .setOutputCol("prediction") \
    .setClassificationModelClass("logreg")\
    .setTol(1e-3)

pipeline = nlp.Pipeline(stages=[document, token, classifier_logreg])

result_logreg = pipeline.fit(train_data).transform(test_data).cache()

result_logreg_df = result_logreg.select('category','prediction.result').toPandas()
result_logreg_df['result'] = result_logreg_df.result.apply(lambda x : x[0])

print (classification_report(result_logreg_df['category'], result_logreg_df['result']))

              precision    recall  f1-score   support

         neg       0.90      0.92      0.91      3383
         pos       0.78      0.74      0.76      1340

    accuracy                           0.87      4723
   macro avg       0.84      0.83      0.83      4723
weighted avg       0.87      0.87      0.87      4723



**Change `maxTokenNgram` parameter with `setMaxTokenNgram()`**

In [23]:
classifier_logreg = medical.DocumentMLClassifierApproach() \
    .setInputCols("token") \
    .setLabelCol("category") \
    .setOutputCol("prediction") \
    .setClassificationModelClass("logreg")\
    .setMaxTokenNgram(2)

pipeline = nlp.Pipeline(stages=[document, token, classifier_logreg])

result_logreg = pipeline.fit(train_data).transform(test_data).cache()

result_logreg_pf = result_logreg.select('category','prediction.result').toPandas()
result_logreg_pf['result'] = result_logreg_pf.result.apply(lambda x : x[0])

print (classification_report(result_logreg_pf['category'], result_logreg_pf['result']))

              precision    recall  f1-score   support

         neg       0.91      0.91      0.91      3383
         pos       0.78      0.78      0.78      1340

    accuracy                           0.88      4723
   macro avg       0.85      0.85      0.85      4723
weighted avg       0.88      0.88      0.88      4723



## Using More Annotators

In [24]:
document = nlp.DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

token = nlp.Tokenizer() \
  .setInputCols(["document"]) \
  .setOutputCol("token")

normalizer = nlp.Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

stopwords_cleaner = nlp.StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

stemmer = nlp.Stemmer() \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("stem")

classifier_svm = medical.DocumentMLClassifierApproach() \
    .setInputCols("stem") \
    .setLabelCol("category") \
    .setOutputCol("prediction") \
    .setClassificationModelClass("svm")\
    .setMaxIter(5)

finisher = nlp.Finisher() \
    .setInputCols(["prediction"]) \
    .setOutputCols(["predictions"]) \
    .setCleanAnnotations(True)\



pipeline = nlp.Pipeline(stages=[document, token, normalizer, stopwords_cleaner, stemmer, classifier_svm, finisher])

model= pipeline.fit(train_data)

In [25]:
sample_df = spark.createDataFrame([["We report two cases of pseudoporphyria caused by naproxen and oxaprozin."],
                                   ["Special attention should be paid when attempting to sample the endometrium in patients with mullerian abnormalities."]]).toDF("text")

result = model.transform(sample_df)
result.show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------+-----------+
|text                                                                                                                |predictions|
+--------------------------------------------------------------------------------------------------------------------+-----------+
|We report two cases of pseudoporphyria caused by naproxen and oxaprozin.                                            |[pos]      |
|Special attention should be paid when attempting to sample the endometrium in patients with mullerian abnormalities.|[neg]      |
+--------------------------------------------------------------------------------------------------------------------+-----------+

